# Project Overview

This project is structured into **four major components**, each addressing a critical phase of **time series forecasting** using the **Rossmann Store Sales dataset**:

🔹 **Part 1**: Data Ingestion, Exploratory Data Analysis (EDA)

We begin by importing the dataset, performing a thorough exploratory analysis, and engineering relevant features to prepare the data for modeling.

🔹 **Part 2**: Feature Engineering and Data Visualization

🔹 **Part 3**: Classical Time Series Analysis and Forecasting

This section focuses on widely adopted forecasting techniques in the data science domain. We will implement and evaluate several standard algorithms, including:

 - **Statistical Models**: ARIMA, SARIMA
    
 - **Ensemble Methods:** XGBoost, LightGBM
    
 - **Facebook Prophet:** A robust model for time series forecasting with built-in seasonality and holiday effects
    
 - **Deep Learning Models:** LSTM, Temporal Fusion Transformers (TFT), N-BEATS

🔹 **Part 4**: Hybrid Time Series Forecasting

This advanced section explores hybrid modeling approaches typically used by experienced data scientists. These models combine the strengths of multiple algorithms to improve forecasting accuracy:

 - **ARIMA + XGBoost**
    
 - **Prophet + LightGBM / XGBoost**
    
 - **Prophet + LSTM**
    
 - **TFT + ARIMA**

## 1. Setup & Imports Libraries
---------------------------------------

In [1]:
import time 

In [2]:
# Step 1: Data Ingestion
print("Step 1: Setup and Import Libraries in progress...")
time.sleep(1)  # Simulate processing time

Step 1: Setup and Import Libraries in progress...


In [3]:
# Data Manipulation & Processing
import os
import holidays
import pandas as pd
import numpy as np
from pathlib import Path
import scipy.stats as stats
from datetime import datetime
from sklearn.preprocessing import *

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format','{:.2f}'.format)

# Warnings
import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')

print("="*60)
print("Rossman Store Sales Time Series Analysis - Part 1")
print("="*60)
print("All libraries imported successfully!")

Rossman Store Sales Time Series Analysis - Part 1
All libraries imported successfully!


In [4]:
print("✅ Setup and Import Liraries completed.\n")

✅ Setup and Import Liraries completed.




## 2. Restore DataFrame
----------------------------

In [5]:
%store -r df_features

## View or Display Dataset

In [6]:
print(df_features.head())

        store  dayofweek       date  sales  customers  open     promo stateholiday  schoolholiday  isholiday  isschoolDay  day  week month  quarter  year  isweekend
982643   1115          2 2013-01-01      0          0     0  No Promo       Public              1       True        False  Tue     1   Jan        1  2013      False
982640   1112          2 2013-01-01      0          0     0  No Promo       Public              1       True        False  Tue     1   Jan        1  2013      False
982639   1111          2 2013-01-01      0          0     0  No Promo       Public              1       True        False  Tue     1   Jan        1  2013      False
982638   1110          2 2013-01-01      0          0     0  No Promo       Public              1       True        False  Tue     1   Jan        1  2013      False
982637   1109          2 2013-01-01      0          0     0  No Promo       Public              1       True        False  Tue     1   Jan        1  2013      False


In [7]:
df_features['date'].min(), df_features['date'].max()

(Timestamp('2013-01-01 00:00:00'), Timestamp('2015-06-30 00:00:00'))

### Top 10 Stores - Customers Performance Analysis

In [8]:
# Simple and robust store analysis
store_avg = df_features.groupby('store')['customers'].mean()
top10 = store_avg.nlargest(10)

print("Top 10 Stores Performance Analysis:")
print("=" * 55)
print(f"{'Rank':<4} {'Store ID':<8} {'Avg Customers':<15} {'% of #1 Store':<12}")
print("-" * 55)

for i, (store, avg_customers) in enumerate(top10.items(), 1):
    pct = (avg_customers / top10.iloc[0]) * 100
    print(f"{i:<4} {store:<8} {avg_customers:>10,.0f}     {pct:>8.1f}%")

print(f"\nSummary Statistics:")
print("-" * 25)
print(f"Total stores analyzed: {len(store_avg):,}")
print(f"Top 10 average: {top10.mean():,.0f} customers")
print(f"Performance range: {top10.max() - top10.min():,.0f} customers")
print(f"Standard deviation: {top10.std():.0f}")

print(f"\nComparative Analysis:")
print("-" * 20)
print(f"Overall store average: {store_avg.mean():.0f} customers")
print(f"Top 10 outperform overall average by: {((top10.mean() - store_avg.mean()) / store_avg.mean()) * 100:.1f}%")

Top 10 Stores Performance Analysis:
Rank Store ID Avg Customers   % of #1 Store
-------------------------------------------------------
1    733           3,403        100.0%
2    262           3,400         99.9%
3    562           3,107         91.3%
4    769           3,072         90.2%
5    1114          2,653         77.9%
6    817           2,605         76.5%
7    1097          2,412         70.9%
8    335           2,391         70.2%
9    259           2,334         68.6%
10   251           2,028         59.6%

Summary Statistics:
-------------------------
Total stores analyzed: 1,115
Top 10 average: 2,740 customers
Performance range: 1,375 customers
Standard deviation: 477

Comparative Analysis:
--------------------
Overall store average: 629 customers
Top 10 outperform overall average by: 335.7%


### Top 10 Stores - Sales Performance Analysis

In [9]:
# Simple and robust sales analysis
store_sales = df_features.groupby('store')['sales'].mean()
top10 = store_sales.nlargest(10)

print("Top 10 Stores Sales Performance Analysis:")
print("=" * 55)
print(f"{'Rank':<4} {'Store ID':<8} {'Avg Sales':<15} {'% of #1 Store':<12}")
print("-" * 55)

for i, (store, avg_sales) in enumerate(top10.items(), 1):
    pct = (avg_sales / top10.iloc[0]) * 100
    print(f"{i:<4} {store:<8} ${avg_sales:>9,.0f}     {pct:>8.1f}%")

print(f"\nSummary Statistics:")
print("-" * 25)
print(f"Total stores analyzed: {len(store_sales):,}")
print(f"Top 10 average: ${top10.mean():,.0f} sales")
print(f"Performance range: ${top10.max() - top10.min():,.0f} sales")
print(f"Standard deviation: ${top10.std():.0f}")

print(f"\nComparative Analysis:")
print("-" * 20)
print(f"Overall store average: ${store_sales.mean():.0f} sales")
print(f"Top 10 outperform overall average by: {((top10.mean() - store_sales.mean()) / store_sales.mean()) * 100:.1f}%")

Top 10 Stores Sales Performance Analysis:
Rank Store ID Avg Sales       % of #1 Store
-------------------------------------------------------
1    262      $   20,684        100.0%
2    817      $   18,106         87.5%
3    562      $   17,985         86.9%
4    1114     $   17,098         82.7%
5    251      $   15,814         76.5%
6    513      $   15,134         73.2%
7    842      $   15,118         73.1%
8    733      $   14,946         72.3%
9    788      $   14,931         72.2%
10   383      $   14,294         69.1%

Summary Statistics:
-------------------------
Total stores analyzed: 1,115
Top 10 average: $16,411 sales
Performance range: $6,390 sales
Standard deviation: $2016

Comparative Analysis:
--------------------
Overall store average: $5750 sales
Top 10 outperform overall average by: 185.4%


### Store Performance Comparison Analysis

In [10]:
# Simple and robust comparison analysis
customers_top10 = df_features.groupby('store')['customers'].mean().nlargest(10)
sales_top10 = df_features.groupby('store')['sales'].mean().nlargest(10)

# Get all unique stores from both top 10 lists
all_stores = list(set(customers_top10.index) | set(sales_top10.index))
all_stores.sort()

print("Store Performance Comparison Analysis:")
print("=" * 80)
print(f"{'Store':<6} {'Customers Rank':<14} {'Avg Customers':<13} {'Sales Rank':<11} {'Avg Sales':<12} {'Performance':<12}")
print("-" * 80)

for store in all_stores:
    # Get customer metrics
    if store in customers_top10.index:
        cust_rank = list(customers_top10.index).index(store) + 1
        cust_avg = customers_top10[store]
    else:
        cust_rank = "Not Top 10"
        cust_avg = df_features[df_features['store'] == store]['customers'].mean()
    
    # Get sales metrics
    if store in sales_top10.index:
        sales_rank = list(sales_top10.index).index(store) + 1
        sales_avg = sales_top10[store]
    else:
        sales_rank = "Not Top 10"
        sales_avg = df_features[df_features['store'] == store]['sales'].mean()
    
    # Determine performance category
    if store in customers_top10.index and store in sales_top10.index:
        performance = "Both Top 10"
    elif store in customers_top10.index:
        performance = "Customers Only"
    else:
        performance = "Sales Only"
    
    # Format output
    cust_rank_str = str(cust_rank) if isinstance(cust_rank, int) else cust_rank
    sales_rank_str = str(sales_rank) if isinstance(sales_rank, int) else sales_rank
    
    print(f"{store:<6} {cust_rank_str:<14} {cust_avg:>9,.0f}     {sales_rank_str:<11} €{sales_avg:>8,.0f}   {performance:<12}")

print(f"\nComparison Summary:")
print("-" * 25)
both_top10 = len(set(customers_top10.index) & set(sales_top10.index))
customers_only = len(set(customers_top10.index) - set(sales_top10.index))
sales_only = len(set(sales_top10.index) - set(customers_top10.index))

print(f"Stores in both top 10: {both_top10}")
print(f"High customers only: {customers_only}")
print(f"High sales only: {sales_only}")
print(f"Total unique stores: {len(all_stores)}")

if both_top10 > 0:
    print(f"\nOverlap rate: {(both_top10 / 10) * 100:.1f}% of top 10 lists")

Store Performance Comparison Analysis:
Store  Customers Rank Avg Customers Sales Rank  Avg Sales    Performance 
--------------------------------------------------------------------------------
251    10                 2,028     5           €  15,814   Both Top 10 
259    9                  2,334     Not Top 10  €  11,484   Customers Only
262    2                  3,400     1           €  20,684   Both Top 10 
335    8                  2,391     Not Top 10  €  13,321   Customers Only
383    Not Top 10         1,825     10          €  14,294   Sales Only  
513    Not Top 10         1,747     6           €  15,134   Sales Only  
562    3                  3,107     3           €  17,985   Both Top 10 
733    1                  3,403     8           €  14,946   Both Top 10 
769    4                  3,072     Not Top 10  €  10,782   Customers Only
788    Not Top 10         1,430     9           €  14,931   Sales Only  
817    6                  2,605     2           €  18,106   Both Top 1

#### Dual Comparison Analysis

In [11]:
# Create the comparison dataframe
customers_top10 = df_features.groupby('store')['customers'].mean().nlargest(10).reset_index().rename(columns={'customers': 'avg_customers'})
sales_top10 = df_features.groupby('store')['sales'].mean().nlargest(10).reset_index().rename(columns={'sales': 'avg_sales'})

comparison_df = pd.merge(
    customers_top10,
    sales_top10,
    on='store',
    how='outer'
).fillna(0)

# Add the top_both flag
comparison_df['top_both'] = (
    comparison_df['store'].isin(customers_top10['store']) &
    comparison_df['store'].isin(sales_top10['store'])
)

# Enhanced display
print("Store Performance Comparison DataFrame:")
print("=" * 70)
print(f"{'Store':<6} {'Avg Customers':<13} {'Avg Sales':<12} {'Top Both':<8}")
print("-" * 70)

for _, row in comparison_df.iterrows():
    customers = int(row['avg_customers']) if row['avg_customers'] > 0 else 0
    sales = int(row['avg_sales']) if row['avg_sales'] > 0 else 0
    top_both = "Yes" if row['top_both'] else "No"
    
    print(f"{int(row['store']):<6} {customers:>9,}     €{sales:>8,}    {top_both:<8}")

print(f"\nSummary:")
print("-" * 15)
total_stores = len(comparison_df)
both_count = comparison_df['top_both'].sum()
customers_only = len(comparison_df[(comparison_df['avg_customers'] > 0) & (~comparison_df['top_both'])])
sales_only = len(comparison_df[(comparison_df['avg_sales'] > 0) & (~comparison_df['top_both'])])

print(f"Total stores in comparison: {total_stores}")
print(f"Top in both categories: {both_count}")
print(f"Top customers only: {customers_only}")
print(f"Top sales only: {sales_only}")
print(f"Overlap percentage: {(both_count/10)*100:.1f}%")

Store Performance Comparison DataFrame:
Store  Avg Customers Avg Sales    Top Both
----------------------------------------------------------------------
251        2,028     €  15,814    Yes     
259        2,333     €       0    No      
262        3,400     €  20,684    Yes     
335        2,390     €       0    No      
383            0     €  14,294    No      
513            0     €  15,133    No      
562        3,106     €  17,984    Yes     
733        3,403     €  14,945    Yes     
769        3,071     €       0    No      
788            0     €  14,930    No      
817        2,605     €  18,105    Yes     
842            0     €  15,117    No      
1097       2,412     €       0    No      
1114       2,652     €  17,097    Yes     

Summary:
---------------
Total stores in comparison: 14
Top in both categories: 6
Top customers only: 4
Top sales only: 4
Overlap percentage: 60.0%


#### Daily Performance Analysis

In [12]:
# Simple and robust daily analysis
day_stats = df_features.groupby("day")[["customers","sales"]].mean().sort_values(by="sales", ascending=False)

print("Daily Performance Analysis:")
print("=" * 60)
print(f"{'Day':<10} {'Avg Customers':<13} {'Avg Sales':<12} {'Sales Rank':<10} {'Customer Rank':<12}")
print("-" * 60)

# Get rankings for both metrics
sales_ranking = day_stats.sort_values(by="sales", ascending=False)
customers_ranking = day_stats.sort_values(by="customers", ascending=False)

for i, (day, row) in enumerate(sales_ranking.iterrows(), 1):
    customers = row['customers']
    sales = row['sales']
    cust_rank = list(customers_ranking.index).index(day) + 1
    
    print(f"{day:<10} {customers:>9,.0f}     €{sales:>8,.0f}    {i:<10} {cust_rank:<12}")

print(f"\nDaily Performance Summary:")
print("-" * 30)
print(f"Best sales day: {sales_ranking.index[0]} (€{sales_ranking.iloc[0]['sales']:,.0f})")
print(f"Best customer day: {customers_ranking.index[0]} ({customers_ranking.iloc[0]['customers']:,.0f} customers)")
print(f"Worst sales day: {sales_ranking.index[-1]} (€{sales_ranking.iloc[-1]['sales']:,.0f})")
print(f"Worst customer day: {customers_ranking.index[-1]} ({customers_ranking.iloc[-1]['customers']:,.0f} customers)")

print(f"\nPerformance Range:")
print("-" * 20)
sales_range = sales_ranking.iloc[0]['sales'] - sales_ranking.iloc[-1]['sales']
customers_range = customers_ranking.iloc[0]['customers'] - customers_ranking.iloc[-1]['customers']
print(f"Sales range: €{sales_range:,.0f}")
print(f"Customer range: {customers_range:,.0f} customers")

# Correlation insight
if sales_ranking.index[0] == customers_ranking.index[0]:
    print(f"\nInsight: {sales_ranking.index[0]} leads in both sales and customer traffic!")
else:
    print(f"\nInsight: Different peak days - Sales: {sales_ranking.index[0]}, Customers: {customers_ranking.index[0]}")

Daily Performance Analysis:
Day        Avg Customers Avg Sales    Sales Rank Customer Rank
------------------------------------------------------------
Mon              813     €   7,798    1          1           
Tue              762     €   7,006    2          2           
Fri              743     €   6,704    3          3           
Wed              721     €   6,536    4          4           
Thu              696     €   6,216    5          5           
Sat              659     €   5,857    6          6           
Sun               36     €     203    7          7           

Daily Performance Summary:
------------------------------
Best sales day: Mon (€7,798)
Best customer day: Mon (813 customers)
Worst sales day: Sun (€203)
Worst customer day: Sun (36 customers)

Performance Range:
--------------------
Sales range: €7,595
Customer range: 777 customers

Insight: Mon leads in both sales and customer traffic!


In [13]:
# Simple and robust monthly analysis
month_stats = df_features.groupby("month")[["customers","sales"]].mean().sort_values(by="sales", ascending=False)

# Month variable is already mapped as Jan, Feb, Mar, etc.

print("Monthly Performance Analysis:")
print("=" * 65)
print(f"{'Month':<10} {'Avg Customers':<13} {'Avg Sales':<12} {'Sales Rank':<10} {'Customer Rank':<12}")
print("-" * 65)

# Get rankings for both metrics
sales_ranking = month_stats.sort_values(by="sales", ascending=False)
customers_ranking = month_stats.sort_values(by="customers", ascending=False)

for i, (month, row) in enumerate(sales_ranking.iterrows(), 1):
    customers = row['customers']
    sales = row['sales']
    cust_rank = list(customers_ranking.index).index(month) + 1
    month_name = month
    
    print(f"{month_name:<10} {customers:>9,.0f}     €{sales:>8,.0f}    {i:<10} {cust_rank:<12}")

print(f"\nMonthly Performance Summary:")
print("-" * 32)
best_sales_month = sales_ranking.index[0]
best_customer_month = customers_ranking.index[0]
worst_sales_month = sales_ranking.index[-1]
worst_customer_month = customers_ranking.index[-1]

print(f"Best sales month: {best_sales_month} (€{sales_ranking.iloc[0]['sales']:,.0f})")
print(f"Best customer month: {best_customer_month} ({customers_ranking.iloc[0]['customers']:,.0f} customers)")
print(f"Worst sales month: {worst_sales_month} (€{sales_ranking.iloc[-1]['sales']:,.0f})")
print(f"Worst customer month: {worst_customer_month} ({customers_ranking.iloc[-1]['customers']:,.0f} customers)")

print(f"\nSeasonal Performance Range:")
print("-" * 25)
sales_range = sales_ranking.iloc[0]['sales'] - sales_ranking.iloc[-1]['sales']
customers_range = customers_ranking.iloc[0]['customers'] - customers_ranking.iloc[-1]['customers']
avg_sales = month_stats['sales'].mean()
avg_customers = month_stats['customers'].mean()

print(f"Sales range: €{sales_range:,.0f}")
print(f"Customer range: {customers_range:,.0f} customers")
print(f"Sales volatility: {(sales_range/avg_sales)*100:.1f}%")
print(f"Customer volatility: {(customers_range/avg_customers)*100:.1f}%")

# Seasonal insights
print(f"\nSeasonal Insights:")
print("-" * 18)
if best_sales_month == best_customer_month:
    print(f"Peak season: {best_sales_month} leads in both sales and customer traffic!")
else:
    print(f"Different peak months - Sales: {best_sales_month}, Customers: {best_customer_month}")

# Identify seasonal patterns
if best_sales_month in ['Nov', 'Dec', 'Jan']:
    print("Pattern: Winter holiday season shows strong performance")
elif best_sales_month in ['Jun', 'Jul', 'Aug']:
    print("Pattern: Summer season shows strong performance")
elif best_sales_month in ['Mar', 'Apr', 'May']:
    print("Pattern: Spring season shows strong performance")
else:
    print("Pattern: Fall season shows strong performance")

Monthly Performance Analysis:
Month      Avg Customers Avg Sales    Sales Rank Customer Rank
-----------------------------------------------------------------
Dec              703     €   6,827    1          1           
Jul              664     €   6,023    2          2           
Nov              654     €   6,008    3          3           
Mar              629     €   5,785    4          8           
Jun              625     €   5,761    5          10          
Apr              631     €   5,739    6          7           
Aug              642     €   5,693    7          4           
Feb              627     €   5,645    8          9           
Sep              634     €   5,570    9          5           
Oct              631     €   5,537    10         6           
May              602     €   5,490    11         11          
Jan              602     €   5,465    12         12          

Monthly Performance Summary:
--------------------------------
Best sales month: Dec (€6,827)
Bes

### Yearly Performance Analysis

In [14]:
# Simple and robust yearly analysis
year_stats = df_features.groupby("year")[["customers","sales"]].mean().sort_values(by="sales", ascending=False)

print("Yearly Performance Analysis:")
print("=" * 70)
print(f"{'Year':<6} {'Avg Customers':<13} {'Avg Sales':<12} {'Sales Rank':<10} {'Customer Rank':<12} {'YoY Growth':<10}")
print("-" * 70)

# Get rankings for both metrics
sales_ranking = year_stats.sort_values(by="sales", ascending=False)
customers_ranking = year_stats.sort_values(by="customers", ascending=False)

# Calculate year-over-year growth (chronological order)
chronological_years = sorted(year_stats.index)
yoy_sales = {}
yoy_customers = {}

for i, year in enumerate(chronological_years):
    if i > 0:
        prev_year = chronological_years[i-1]
        sales_growth = ((year_stats.loc[year, 'sales'] - year_stats.loc[prev_year, 'sales']) / year_stats.loc[prev_year, 'sales']) * 100
        customer_growth = ((year_stats.loc[year, 'customers'] - year_stats.loc[prev_year, 'customers']) / year_stats.loc[prev_year, 'customers']) * 100
        yoy_sales[year] = sales_growth
        yoy_customers[year] = customer_growth
    else:
        yoy_sales[year] = 0
        yoy_customers[year] = 0

for i, (year, row) in enumerate(sales_ranking.iterrows(), 1):
    customers = row['customers']
    sales = row['sales']
    cust_rank = list(customers_ranking.index).index(year) + 1
    growth = yoy_sales.get(year, 0)
    growth_str = f"{growth:+.1f}%" if growth != 0 else "Baseline"
    
    print(f"{year:<6} {customers:>9,.0f}     €{sales:>8,.0f}    {i:<10} {cust_rank:<12} {growth_str:<10}")

print(f"\nYearly Performance Summary:")
print("-" * 30)
best_sales_year = sales_ranking.index[0]
best_customer_year = customers_ranking.index[0]
worst_sales_year = sales_ranking.index[-1]
worst_customer_year = customers_ranking.index[-1]

print(f"Best sales year: {best_sales_year} (€{sales_ranking.iloc[0]['sales']:,.0f})")
print(f"Best customer year: {best_customer_year} ({customers_ranking.iloc[0]['customers']:,.0f} customers)")
print(f"Worst sales year: {worst_sales_year} (€{sales_ranking.iloc[-1]['sales']:,.0f})")
print(f"Worst customer year: {worst_customer_year} ({customers_ranking.iloc[-1]['customers']:,.0f} customers)")

print(f"\nMulti-Year Performance Trends:")
print("-" * 32)
total_years = len(year_stats)
sales_range = sales_ranking.iloc[0]['sales'] - sales_ranking.iloc[-1]['sales']
customers_range = customers_ranking.iloc[0]['customers'] - customers_ranking.iloc[-1]['customers']

print(f"Years analyzed: {total_years}")
print(f"Sales range: €{sales_range:,.0f}")
print(f"Customer range: {customers_range:,.0f} customers")

# Calculate overall trend
if len(chronological_years) > 1:
    first_year = chronological_years[0]
    last_year = chronological_years[-1]
    overall_sales_growth = ((year_stats.loc[last_year, 'sales'] - year_stats.loc[first_year, 'sales']) / year_stats.loc[first_year, 'sales']) * 100
    overall_customer_growth = ((year_stats.loc[last_year, 'customers'] - year_stats.loc[first_year, 'customers']) / year_stats.loc[first_year, 'customers']) * 100
    
    print(f"Overall sales trend ({first_year}-{last_year}): {overall_sales_growth:+.1f}%")
    print(f"Overall customer trend ({first_year}-{last_year}): {overall_customer_growth:+.1f}%")

print(f"\nBusiness Insights:")
print("-" * 18)
if best_sales_year == best_customer_year:
    print(f"Peak performance: {best_sales_year} excelled in both sales and customer traffic!")
else:
    print(f"Different peak years - Sales: {best_sales_year}, Customers: {best_customer_year}")

# Growth trend analysis
if len(chronological_years) > 1:
    recent_growth = yoy_sales.get(chronological_years[-1], 0)
    if recent_growth > 5:
        print(f"Trend: Strong recent growth of {recent_growth:+.1f}% in latest year")
    elif recent_growth < -5:
        print(f"Trend: Concerning decline of {recent_growth:+.1f}% in latest year")
    else:
        print(f"Trend: Stable performance with {recent_growth:+.1f}% change in latest year")

Yearly Performance Analysis:
Year   Avg Customers Avg Sales    Sales Rank Customer Rank YoY Growth
----------------------------------------------------------------------
2014         643     €   5,833    1          1            +3.1%     
2015         621     €   5,833    2          3            -0.0%     
2013         629     €   5,659    3          2            Baseline  

Yearly Performance Summary:
------------------------------
Best sales year: 2014 (€5,833)
Best customer year: 2014 (643 customers)
Worst sales year: 2013 (€5,659)
Worst customer year: 2015 (621 customers)

Multi-Year Performance Trends:
--------------------------------
Years analyzed: 3
Sales range: €175
Customer range: 22 customers
Overall sales trend (2013-2015): +3.1%
Overall customer trend (2013-2015): -1.3%

Business Insights:
------------------
Peak performance: 2014 excelled in both sales and customer traffic!
Trend: Stable performance with -0.0% change in latest year


### Yearly Performance Analysis - Simple and robust promotion & day analysis

In [15]:
# Simple and robust promotion & day analysis
promo_day = df_features.groupby(["promo", "day"])[["customers", "sales"]].mean().sort_values("sales", ascending=False)

print("Promotion & Day of Week Analysis:")
print("=" * 70)
print(f"{'Promo':<6} {'Day':<10} {'Avg Customers':<13} {'Avg Sales':<12} {'Rank':<6}")
print("-" * 70)

for i, ((promo, day), row) in enumerate(promo_day.iterrows(), 1):
    promo_str = "Yes" if promo == 1 else "No"
    print(f"{promo_str:<6} {day:<10} {row['customers']:>9,.0f}     €{row['sales']:>8,.0f}    {i:<6}")

print(f"\nPromotion Impact by Day:")
print("-" * 25)
print(f"{'Day':<10} {'No Promo':<12} {'With Promo':<12} {'Lift':<8}")
print("-" * 45)

for day in sorted(df_features['day'].unique()):
    no_promo = df_features[(df_features['promo'] == 0) & (df_features['day'] == day)]['sales'].mean()
    with_promo = df_features[(df_features['promo'] == 1) & (df_features['day'] == day)]['sales'].mean()
    
    if not pd.isna(no_promo) and not pd.isna(with_promo):
        lift = ((with_promo - no_promo) / no_promo) * 100
        print(f"{day:<10} €{no_promo:>7,.0f}     €{with_promo:>8,.0f}     {lift:>5.1f}%")

print(f"\nKey Insights:")
print("-" * 15)
best_combo = promo_day.index[0]
overall_lift = ((df_features[df_features['promo']==1]['sales'].mean() - df_features[df_features['promo']==0]['sales'].mean()) / df_features[df_features['promo']==0]['sales'].mean()) * 100

print(f"Best combination: {'Promo' if best_combo[0]==1 else 'No Promo'} on {best_combo[1]}")
print(f"Overall promo lift: {overall_lift:+.1f}%")
print(f"Top sales: €{promo_day.iloc[0]['sales']:,.0f}")

Promotion & Day of Week Analysis:
Promo  Day        Avg Customers Avg Sales    Rank  
----------------------------------------------------------------------
No     Mon              939     €   9,709    1     
No     Tue              837     €   8,226    2     
No     Wed              786     €   7,541    3     
No     Thu              774     €   7,242    4     
No     Fri              766     €   7,169    5     
No     Fri              717     €   6,180    6     
No     Sat              659     €   5,857    7     
No     Tue              675     €   5,608    8     
No     Mon              666     €   5,568    9     
No     Wed              648     €   5,404    10    
No     Thu              608     €   5,063    11    
No     Sun               36     €     203    12    

Promotion Impact by Day:
-------------------------
Day        No Promo     With Promo   Lift    
---------------------------------------------

Key Insights:
---------------
Best combination: No Promo on Mon
Overall pr

# ✨ Reusable Functions 
-----------
It is a best practice wrap analysis impact and others into a reusable utility function so you can generate, reuse and export charts seamlessly from any part of your notebook or script. These best practices can make your code clean, powerful, and highly maintainable—whether you're working with data cleaning, aggregation, visualization, or export routines.


# 🔧 Generalized Reusable Function

In [16]:
def summarize_by_time(df, time_col, metrics, agg_func='mean', rename_prefix='avg', sort_by=None, ascending=False):
    """
    General-purpose time-based aggregation summary.

    Parameters:
    - df: pandas DataFrame
    - time_col: column to group by (e.g., 'year', 'month', 'dayofweek')
    - metrics: list of metric columns to aggregate (e.g., ['sales', 'customers'])
    - agg_func: aggregation function (default: 'mean')
    - rename_prefix: prefix added to aggregated column names
    - sort_by: column to sort by (e.g., 'avg_sales')
    - ascending: sorting direction (default: descending)

    Returns:
    - Aggregated and optionally sorted DataFrame
    """
    summary = (
        df.groupby(time_col)[metrics]
        .agg(agg_func)
        .reset_index()
        .rename(columns={metric: f"{rename_prefix}_{metric}" for metric in metrics})
    )

    if sort_by:
        summary = summary.sort_values(by=sort_by, ascending=ascending)

    return summary


In [17]:
day_analysis = summarize_by_time(
    df=df_features,
    time_col='day',
    metrics=['customers', 'sales'],
    agg_func='mean',
    rename_prefix='avg',
    sort_by='avg_sales',
    ascending=False
)

print(day_analysis)

   day  avg_customers  avg_sales
1  Mon         812.93    7797.64
5  Tue         761.86    7005.52
0  Fri         742.53    6703.50
6  Wed         721.20    6536.45
4  Thu         695.78    6216.11
2  Sat         658.76    5856.78
3  Sun          35.58     202.62


In [18]:
month_analysis = summarize_by_time(
    df=df_features,
    time_col='month',
    metrics=['customers', 'sales'],
    agg_func='mean',
    rename_prefix='avg',
    sort_by='avg_sales',
    ascending=False
)

print(month_analysis)


   month  avg_customers  avg_sales
2    Dec         703.07    6826.61
5    Jul         663.59    6022.61
9    Nov         654.15    6008.11
7    Mar         629.40    5784.58
6    Jun         624.79    5760.96
0    Apr         630.61    5738.87
1    Aug         642.50    5693.02
3    Feb         626.72    5645.25
11   Sep         634.44    5570.25
10   Oct         631.10    5537.04
8    May         601.99    5489.64
4    Jan         601.62    5465.40


In [19]:
year_analysis = summarize_by_time(
    df=df_features,
    time_col='year',
    metrics=['customers', 'sales'],
    agg_func='mean',
    rename_prefix='avg',
    sort_by='avg_sales',
    ascending=False
)

print(year_analysis)


   year  avg_customers  avg_sales
1  2014         643.27    5833.29
2  2015         620.84    5832.95
0  2013         629.04    5658.53


# 🌟 Advantages

- Reusable across 'year', 'month', 'dayofweek', etc.

- Easy to change aggregation type ('sum', 'median', etc.)

- Consistent naming and sorting

- Makes your code far more modular for dashboards or reporting

# Why Reusability Matters
-------------------------
 - 💡 Scalability: You can plug your functions into larger pipelines or production environments without rewrites.
 - 🛠️ Maintainability: A bug fix in one utility can instantly improve multiple workflows.
 - 🚀 Efficiency: Spend less time rewriting logic and more time interpreting results.

# Why This Matters for Rossmann Store Sales
 - We’ll likely repeat the same aggregations or visualizations across hundreds of stores.
 - Promos, holidays, and weekday patterns demand consistent filtering and analysis.
 - Modular functions help you prototype insights fast, scale across stores, and iterate smoothly.
     

---------------------------

In [ ]:
print("✅ Data Ingestion and Exploratory Data Analysis completed successfully!")
print(f"🗓️ Analysis Date: {bold_start}{pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}{bold_end}")

--------------------------------

In [ ]:
# End analysis
analysis_end = pd.Timestamp.now()
duration = analysis_end - analysis_begin

# Final summary print
print("\n📋 Analysis Summary")
print(f"🟢 Begin Date: {bold_start}{analysis_begin.strftime('%Y-%m-%d %H:%M:%S')}{bold_end}")
print(f"✅ End Date:   {bold_start}{analysis_end.strftime('%Y-%m-%d %H:%M:%S')}{bold_end}")
print(f"⏱️ Duration:   {bold_start}{str(duration)}{bold_end}")

-------------------------
## Project Design Rationale: Notebook Separation

To promote **clarity, maintainability, and scalability** within the project, **data engineering** and **visualization tasks** are intentionally separated into distinct notebooks. This modular approach prevents the accumulation of excessive code in a single notebook, making it easier to **debug, update, and collaborate across different stages of the workflow**. By isolating data transformation logic from visual analysis, **each notebook remains focused and purpose-driven**, ultimately **enhancing the overall efficiency and readability of the project**.